In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from torchvision import models

# Dataset paths (ensure these directories exist in your Google Drive)
train_path = os.path.join('/content/drive/MyDrive/FYP/dataset', 'train')
valid_path = os.path.join('/content/drive/MyDrive/FYP/dataset', 'validation')
test_path = os.path.join('/content/drive/MyDrive/FYP/dataset', 'test')

# Configurations
batch_size = 16
num_classes = 4  # happy, angry, sad, neutral
learning_rate = 0.001
num_epochs = 100
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.ToTensor(),
])

# Load datasets
train_data = ImageFolder(train_path, transform=transform)
valid_data = ImageFolder(valid_path, transform=transform)
test_data = ImageFolder(test_path, transform=transform)

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

# Load pre-trained model and modify classifier
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Early stopping parameters
best_valid_loss = float('inf')
patience = 5  # Number of epochs to wait for improvement
patience_counter = 0

# Training loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    train_accuracy = 100 * correct / total
    print(f"Epoch [{epoch+1}/{num_epochs}] - Loss: {total_loss:.4f} - Train Accuracy: {train_accuracy:.2f}%")

    # Validation loop
    model.eval()
    val_loss = 0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for inputs, labels in valid_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, predicted = outputs.max(1)
            val_total += labels.size(0)
            val_correct += predicted.eq(labels).sum().item()

    val_accuracy = 100 * val_correct / val_total
    print(f"Validation Accuracy: {val_accuracy:.2f}%")

    # Early stopping check
    if val_loss < best_valid_loss:
        best_valid_loss = val_loss
        patience_counter = 0
        torch.save(model.state_dict(), "best_emotion_model.pth")  # Save the model with the best validation loss
    else:
        patience_counter += 1
        if patience_counter >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break

# Final testing loop
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

test_accuracy = 100 * correct / total
print(f"\nFinal Test Accuracy: {test_accuracy:.2f}%")

# Optional: Save the final trained model
torch.save(model.state_dict(), "emotion_model_final.pth")


Epoch [1/100] - Loss: 161.0500 - Train Accuracy: 31.24%
Validation Accuracy: 38.56%
Epoch [2/100] - Loss: 131.9082 - Train Accuracy: 47.05%
Validation Accuracy: 51.06%
Epoch [3/100] - Loss: 103.2669 - Train Accuracy: 62.41%
Validation Accuracy: 70.74%
Epoch [4/100] - Loss: 75.7856 - Train Accuracy: 74.60%
Validation Accuracy: 68.09%
Epoch [5/100] - Loss: 59.4234 - Train Accuracy: 80.56%
Validation Accuracy: 88.30%
Epoch [6/100] - Loss: 38.2428 - Train Accuracy: 87.93%
Validation Accuracy: 86.70%
Epoch [7/100] - Loss: 36.6644 - Train Accuracy: 88.04%
Validation Accuracy: 91.49%
Epoch [8/100] - Loss: 24.5777 - Train Accuracy: 92.23%
Validation Accuracy: 90.43%
Epoch [9/100] - Loss: 24.8934 - Train Accuracy: 92.74%
Validation Accuracy: 96.54%
Epoch [10/100] - Loss: 13.1031 - Train Accuracy: 96.60%
Validation Accuracy: 94.15%
Epoch [11/100] - Loss: 26.0680 - Train Accuracy: 91.78%
Validation Accuracy: 94.68%
Epoch [12/100] - Loss: 16.1704 - Train Accuracy: 95.12%
Validation Accuracy: 94.41